# My first attempt!

`This was my first attempt at using LSTM networks in keras to predict lyrics. I used the lyrics from In the Aeroplane Over the Sea as my data. This initial attempt was not very successful in terms of making a model the outputs lyrics, but it was instrumental to me developing an understanding of LSTM RNNs and how to structure the data for them.`

In [7]:
import pandas as pd
import numpy as np

In [12]:
def read_lyrics(file):
    return pd.read_csv(file, sep='|', header=None, error_bad_lines=False)

In [13]:
alt_country_df       = read_lyrics('songs/alt_countrysong_lyrics.psv')
alt_rock_df          = read_lyrics('songs/alt_rocksong_lyrics.psv')
experimental_rock_df = read_lyrics('songs/experimental_rocksong_lyrics.psv')
hip_hop_df           = read_lyrics('songs/hip_hipsong_lyrics.psv')
pop_songs_df         = read_lyrics('songs/popsong_lyrics.psv')
rock_songs_df        = read_lyrics('songs/Rocksong_lyrics.psv')
folk = read_lyrics('data/song_lyrics.psv')

b'Skipping line 1157: expected 9 fields, saw 10\nSkipping line 1158: expected 9 fields, saw 10\nSkipping line 1159: expected 9 fields, saw 10\nSkipping line 1160: expected 9 fields, saw 10\nSkipping line 1161: expected 9 fields, saw 10\nSkipping line 1162: expected 9 fields, saw 10\nSkipping line 1163: expected 9 fields, saw 10\nSkipping line 1164: expected 9 fields, saw 10\nSkipping line 1166: expected 9 fields, saw 17\nSkipping line 1906: expected 9 fields, saw 10\nSkipping line 7590: expected 9 fields, saw 10\nSkipping line 7601: expected 9 fields, saw 89\nSkipping line 12820: expected 9 fields, saw 15\nSkipping line 12938: expected 9 fields, saw 51\nSkipping line 13082: expected 9 fields, saw 14\n'
b'Skipping line 2237: expected 9 fields, saw 10\n'
b'Skipping line 12297: expected 9 fields, saw 19\nSkipping line 18442: expected 9 fields, saw 11\n'


In [36]:
list_of_dfs = [alt_country_df, alt_rock_df, experimental_rock_df, hip_hop_df, pop_songs_df, rock_songs_df]
list_of_names = ['alt_country', 'alt_rock', 'experimental_rock', 'hip_hop', 'pop_songs', 'rock_songs']

In [37]:
def is_genre(cell):
    return 1

In [41]:
for count, df in enumerate(list_of_dfs):
    name = list_of_names[count]
    df['is_' + name] = df[3].apply(is_genre)

In [43]:
df1 = list_of_dfs[0]
for next_df in list_of_dfs[1:]:
    df1 = pd.concat((df1, next_df))

In [47]:
df = pd.concat((folk, df1))

/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [50]:
df.fillna(0, inplace=True)

In [51]:
df.to_csv('all_songs.csv', index=False)

In [215]:
import sys
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Dropout
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
import collections

In [166]:
folk[7]

0       [hook] n I've been to a minor place n And I ca...
1       Today was one where, lost in thought n I reall...
2       Well you're my friend n (That's what you told ...
3       I like to have a good time n Any of my friends...
4       I am here, right here n Where God puts non asu...
5       Fire-burned and blew out flowers n Showing me ...
6       Sing a song of madeleine-mary n A tune that al...
7       Your little feet n Your sharp teeth n The way ...
8       Tonight my eyes were hurting much n As I had s...
9       Black, you are my enemy n And I cannot get clo...
10      Darling n I can stay awake all night n And I w...
11      [Verse 1] n When you were young, you were the ...
12      Part Two n [Verse 1] n I love you, Jesus Chris...
13      [Verse 1] n What a beautiful face n I have fou...
14      [Verse 1] n Two headed boy n All floating in g...
15                                         [Instrumental]
16      [Verse 1] n The only girl I've ever loved n Wa...
17      [Intro

In [168]:
longest = 0
for line in nmh_lyrics[0]:
    if len(line.split()) > longest:
        longest = len(line.split())

In [62]:
nmh_lyrics = pd.read_table('nmh_lyrics.txt', header=None)

In [171]:
data = []

for line in nmh_lyrics[0]:
    x = ((line.lower() + '<eos>').split())
    for word in x:
        data.append(word)

counter = collections.Counter(data)

count_paris = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

words, counts = list(zip(*count_paris))

word_to_id = dict(zip(words, range(len(words))))

word_to_vec = [word_to_id[word] if word in word_to_id else word_to_id['unknown'] for word in data]

reverse_dict = dict(zip(range(len(words)), words))

In [194]:
n_vocab = len(word_to_id)

In [304]:
X = np.zeros((n_patterns, seq_length, n_vocab), dtype=np.bool)
y = np.zeros((n_patterns, n_vocab), dtype=np.bool)

for i, sentence in enumerate(dataX):
    for t, word in enumerate(sentence):
        X[i, t, word] = 1
    y[i, word] = 1

In [311]:
model_0 = Sequential()
model_0.add(LSTM(512, return_sequences=True, input_shape=(seq_length, n_vocab)))
model_0.add(Dropout(0.2))
model_0.add(LSTM(512, return_sequences=False))
model_0.add(Dropout(0.2))
model_0.add(Dense(len(words)))
#model.add(Dense(1000))
model_0.add(Activation('softmax'))

model_0.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [313]:
model_0.fit(X, y, batch_size=64, epochs=20)

Epoch 1/20
2519/2519 [==============================] - 52s 20ms/step - loss: 5.8090
Epoch 2/20
2519/2519 [==============================] - 48s 19ms/step - loss: 5.4257
Epoch 3/20
2519/2519 [==============================] - 51s 20ms/step - loss: 5.4018
Epoch 4/20
2519/2519 [==============================] - 55s 22ms/step - loss: 5.3087
Epoch 5/20
2519/2519 [==============================] - 49s 19ms/step - loss: 5.2217
Epoch 6/20
2519/2519 [==============================] - 48s 19ms/step - loss: 5.0233
Epoch 7/20
2519/2519 [==============================] - 54s 21ms/step - loss: 4.8431
Epoch 8/20
2519/2519 [==============================] - 49s 19ms/step - loss: 4.7970
Epoch 9/20
2519/2519 [==============================] - 48s 19ms/step - loss: 4.5731
Epoch 10/20
2519/2519 [==============================] - 56s 22ms/step - loss: 4.4187
Epoch 11/20
2519/2519 [==============================] - 51s 20ms/step - loss: 4.2697
Epoch 12/20
2519/2519 [==============================] - 53s 21

In [463]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature 
    dist = np.exp(a)/np.sum(np.exp(a))
    choices = range(len(a))
    return np.random.choice(choices, p=dist)

In [468]:
start = np.random.randint(0, len(word_to_vec) - 1)
pattern = word_to_vec[start:start + seq_length]

print(' '.join([reverse_dict[x] for x in pattern]))


for i in range(1000):
    x = np.zeros((1, len(pattern), 669))
    for c, word in enumerate(pattern):
        x[0, c, word] = 1
    
    prediction = model_0.predict(x, verbose=0)[0]
    
    
    
    index = sample(prediction, diversity)
    
    result = reverse_dict[index]
    with open('512_hidden_two_LTSM_layers_first_attempt.txt', 'a+') as f:
        print(result, file=f)
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

the life we used to love <eos> just to keep ourselves <eos> at least enough to carry on <eos> [verse
[versewillisleftsongsateveryladoallyouryouryouryourwillbodyagoherheraayouyouyouyouyouyouyouyouyouyoui[verseyouyouyouyouyouyousheyouyouiiyouryouryouryourweyouyouthatyouyouyouyouiwillyouryourwilli'mlayallalliyouryourinlalalawho'sheremeforinin[versewillisdo,measyouyou[verseyoudeedeewithdeeshe[versethethetheyourthatboyforyouyouyouwillwillwilldohereasherheryouyouyouyouwill[versewillwould[instrumental]<eos>fetusesdeeiyouryouryouryouraherboymewewithitototototoamyonlyuntilmeher[verseiweawillyouracouldwill[verse[verseyouyouyouawillyouryourbeaparttotothetheyourwill[versetotoyouyouyouyouyouthatthethewillpluckbendwithwith<eos><eos><eos><eos><eos>youryourbeearthboymeininyouryouryouryourawillaherfromsongdoinallallallyouryourwilldon'tleftforwiththethethethethethewillheadleftforall<eos><eos><eos><eos>thatwewouldwouldwouldwhereallallwithyouryour[versewillayouryour[versewassingrosestotothatyouyouyouryour

In [286]:
seq_length = 20
dataX, dataY = [], []
for i in range(0, len(data) - seq_length, 1):
    seq_in = word_to_vec[i:i + seq_length]
    seq_out = word_to_vec[i + seq_length]
    dataX.append(seq_in)
    dataY.append(seq_out)
n_patterns = len(dataX)

In [287]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
X = X / n_vocab
y = to_categorical(dataY)

In [271]:
model = Sequential()
# model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(TimeDistributed(Dense(669)))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [272]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False)
callbacks_list = [checkpoint]

In [273]:
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
2514/2514 [==============================] - 16s 6ms/step - loss: 5.7712 - categorical_accuracy: 0.1289

Epoch 00001: loss improved from inf to 5.77121, saving model to weights-improvement-01-5.7712.hdf5
Epoch 2/20
2514/2514 [==============================] - 16s 6ms/step - loss: 5.4073 - categorical_accuracy: 0.1313

Epoch 00002: loss improved from 5.77121 to 5.40726, saving model to weights-improvement-02-5.4073.hdf5
Epoch 3/20
2514/2514 [==============================] - 16s 7ms/step - loss: 5.3686 - categorical_accuracy: 0.1313

Epoch 00003: loss improved from 5.40726 to 5.36857, saving model to weights-improvement-03-5.3686.hdf5
Epoch 4/20
2514/2514 [==============================] - 14s 6ms/step - loss: 5.3650 - categorical_accuracy: 0.1313

Epoch 00004: loss improved from 5.36857 to 5.36503, saving model to weights-improvement-04-5.3650.hdf5
Epoch 5/20
2514/2514 [==============================] - 15s 6ms/step - loss: 5.3575 - categorical_accuracy: 0.1313

Epoch 00005:

In [274]:
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]


for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/669
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = reverse_dict[index]
#     with open('fourth_attempt.txt', 'a+') as f:
#         print(result, file=f)
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>

In [145]:
history = model.fit(x, y, epochs=100)

Epoch 1/100
1/1 [==============================] - 17s 17s/step - loss: 6.5098 - categorical_accuracy: 3.9386e-04
Epoch 2/100
1/1 [==============================] - 16s 16s/step - loss: 6.4637 - categorical_accuracy: 0.0087
Epoch 3/100
1/1 [==============================] - 17s 17s/step - loss: 6.4146 - categorical_accuracy: 0.1315
Epoch 4/100
1/1 [==============================] - 16s 16s/step - loss: 6.3596 - categorical_accuracy: 0.1315
Epoch 5/100
1/1 [==============================] - 16s 16s/step - loss: 6.2979 - categorical_accuracy: 0.1315
Epoch 6/100
1/1 [==============================] - 17s 17s/step - loss: 6.2310 - categorical_accuracy: 0.1315
Epoch 7/100
1/1 [==============================] - 16s 16s/step - loss: 6.1612 - categorical_accuracy: 0.1315
Epoch 8/100
1/1 [==============================] - 16s 16s/step - loss: 6.0902 - categorical_accuracy: 0.1315
Epoch 9/100
1/1 [==============================] - 16s 16s/step - loss: 6.0194 - categorical_accuracy: 0.1315
Epoch 

Epoch 75/100
1/1 [==============================] - 17s 17s/step - loss: 5.2214 - categorical_accuracy: 0.1315
Epoch 76/100
1/1 [==============================] - 16s 16s/step - loss: 5.2174 - categorical_accuracy: 0.1315
Epoch 77/100
1/1 [==============================] - 17s 17s/step - loss: 5.2131 - categorical_accuracy: 0.1315
Epoch 78/100
1/1 [==============================] - 18s 18s/step - loss: 5.2086 - categorical_accuracy: 0.1315
Epoch 79/100
1/1 [==============================] - 17s 17s/step - loss: 5.2040 - categorical_accuracy: 0.1315
Epoch 80/100
1/1 [==============================] - 16s 16s/step - loss: 5.1992 - categorical_accuracy: 0.1315
Epoch 81/100
1/1 [==============================] - 17s 17s/step - loss: 5.1942 - categorical_accuracy: 0.1315
Epoch 82/100
1/1 [==============================] - 17s 17s/step - loss: 5.1892 - categorical_accuracy: 0.1315
Epoch 83/100
1/1 [==============================] - 16s 16s/step - loss: 5.1840 - categorical_accuracy: 0.1315
E

In [18]:
df.columns = ['song_id', 'album_id', 'artist', 'album', 'album_href', 'song', 'track_count', 'lyrics',
              'song_href', 'is_folk', 'is_folk_rock', 'is_indie_folk', 'is_alt_country', 'is_alt_rock',
              'is_experimental_rock', 'is_hip_hop', 'is_pop_songs', 'is_rock_songs']

df['song_id'] = df.index

df.to_csv('all_songs.csv', index=False)